In [ ]:
#!pip install psycopg2-binary #for Michelle

In [ ]:
import numpy as np
import pandas as pd
import os, glob
import sqlalchemy
import sqlite3
from sqlite3 import Error
import sklearn as sk
from sklearn.linear_model import LogisticRegression
from sklearn import svm, metrics
#from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import math
import gzip
import csv
import matplotlib.pyplot as plt
import psycopg2

# Connecting to mimic postgres database

In [ ]:
from sqlalchemy import create_engine

# Postgres username, password, database name
# @team - you'll need to change this according to what your Postgres info is under the Properties tab of your server in PgAdmin

# Michelle's info
#POSTGRES_ADDRESS = '127.0.0.1'
#POSTGRES_PORT= '5432'
#POSTGRES_USERNAME= 'postgres'
#POSTGRES_PASSWORD= 'password'
#POSTGRES_DBNAME = 'mimic'
#postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))

# Paroma's info 
POSTGRES_ADDRESS = 'localhost'   #127.0.0.1
POSTGRES_PORT= '5432'
POSTGRES_USERNAME= 'MacUser'
POSTGRES_PASSWORD= ''
POSTGRES_DBNAME = 'mimic'

# putting in info
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}?options=-csearch_path%3Dmimiciii'.format(username=POSTGRES_USERNAME,password=POSTGRES_PASSWORD,ipaddress=POSTGRES_ADDRESS,port=POSTGRES_PORT,dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)


In [ ]:
# path with all un-normalized data - chartevents, lab events, procedure events
path = "/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/DATA/"

# For patient ids that meet our inclusion/exclusion criteria
# loading all files from path with labevent name
cohort_files = glob.glob(os.path.join(path, "matched_ids_03032021.csv"))
#print(cohort_files)

# initializing data frame
cohort_df = []

# loading all files into data frame
for f in cohort_files:
    df = pd.read_csv(f)
    df['file'] = f.split('/')[-1]
    cohort_df.append(df)

# concatenating files into single data frame    
cohort = pd.concat(cohort_df, ignore_index=True)
print(cohort.head(5))

C:\Users\mnigc\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
path1 = "/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/Un-normalized_matched_patient_id_Data/"

# For lab events
# loading all files from path with labevent name
cohort_files1 = glob.glob(os.path.join(path1, "lab_events*"))
#print(cohort_files)

# initializing data frame
cohort_df1 = []

# loading all files into data frame
for f1 in cohort_files1:
    df1 = pd.read_csv(f1)
    df1['file'] = f1.split('/')[-1]
    cohort_df1.append(df1)

# concatenating files into single data frame    
cohort1 = pd.concat(cohort_df1, ignore_index=True)
print(cohort1.head(5))


In [ ]:
#  getting subject ids from cohort files
#patient_ids = cohort['subject_id']
dead_ids = cohort['Subject_id_dead'];
alive_ids = cohort['Subject_id_alive'];

dead_ids = dead_ids[~np.isnan(dead_ids)]; # 483 dead patients
alive_ids = alive_ids[~np.isnan(alive_ids)]; # 2114 alive patients

unique_ids = pd.concat([alive_ids,dead_ids])
unique_ids.drop_duplicates()
print(unique_ids) #2597 patient ids


2596

# Feature extraction and cleaning by group

In [ ]:
# 03/04/21: Find prevalence of itemid
# Michelle and Zhiwei's definition:  "how often does this lab come up"

# Export prevalence of lab itemids into csv 
item_ids = pd.read_sql_query('''SELECT itemid, label, fluid 
FROM d_labitems WHERE
label LIKE '%Albumin%' OR
label LIKE '%Anion%' OR 
label LIKE '%Bilirubin%' OR
label LIKE '%Chloride%' OR 
label LIKE '%Count%'OR
label LIKE '%Creatinine%' OR
label LIKE '%Glucose%' OR
label LIKE '%Hemoglobin%' OR 
label LIKE '%Lactate%' OR 
label LIKE '%Oxygen Saturation%' OR
label LIKE '%pH%' OR
label LIKE '%Platelet%' OR
label LIKE '%Potassium%' OR  
label LIKE '%Sodium%' OR 
label LIKE '%Temperature%' 
AND label NOT LIKE '%Reticulocyte%';''',cnx)
item_ids = item_ids['itemid']

# Make empty pandas dataframe for prevalence
prevalence = pd.DataFrame(index=item_ids, columns = ['prevalence'])
prevalence = prevalence.fillna(0)
#print(prevalence)

# For every itemid find the prevalence
# There will be an intermediate dataframe to help determine prevalence
# NEVER loop with pandas dataframe
# need to link icustays to lab id
# lowest 40%, highest 80%

lab_events = cohort1;

for i in item_ids:
    temp_lab = lab_events[lab_events['itemid'] == i]
    temp_lab = temp_lab['subject_id'].drop_duplicates()
    prev = temp_lab.shape[0]/unique_ids.shape[0]
    prevalence.loc[i, 'prevalence'] = prev
print(prevalence)    
    
# Export prevalence dataframe to csv file
prevalence.to_csv(r'/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/DATA/prevalence_lab_itemids.csv', index=True)


In [ ]:
# Export label and fluids data into csv
item_ids1 = pd.read_sql_query('''SELECT label, fluid, category
FROM d_labitems WHERE
label LIKE '%Albumin%' OR
label LIKE '%Anion%' OR 
label LIKE '%Bilirubin%' OR
label LIKE '%Chloride%' OR 
label LIKE '%Count%'OR
label LIKE '%Creatinine%' OR
label LIKE '%Glucose%' OR
label LIKE '%Hemoglobin%' OR 
label LIKE '%Lactate%' OR 
label LIKE '%Oxygen Saturation%' OR
label LIKE '%pH%' OR
label LIKE '%Platelet%' OR
label LIKE '%Potassium%' OR  
label LIKE '%Sodium%' OR 
label LIKE '%Temperature%' 
AND label NOT LIKE '%Reticulocyte%';''',cnx)

item_ids1.to_csv(r'/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/DATA/lab_labels_fluids_category.csv', index=False)

print('Done exporting csv!')

In [ ]:
## Load in itemid, prevalence data from prevalence_lab_itemids.csv file
labflu = pd.read_csv(r"/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/DATA/lab_labels_fluids_category.csv")
prev = pd.read_csv(r"/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/DATA/prevalence_lab_itemids.csv")

# Join label, fluid, itemid, prevalence into one dataframe
prevalence_frame = pd.concat([labflu, prev], axis=1, join='inner')
#print(prevalence_frame)

# Take out rows where prevalence < 0.7
prevalence_filtered = prevalence_frame[prevalence_frame['prevalence'] >= 0.7]
the_item_ids = prevalence_filtered['itemid']
print(prevalence_filtered)

In [ ]:
# Find min, max, mean, stdev for each patient's lab event
# csv per feature set
    # make sure in the same order 
    # join later
# This should all be really fast
    # each lab should take a couple seconds

for j in the_item_ids:
    temp_lab1 = lab_events[lab_events['itemid'] == j].reset_index(drop=True) 
    temp_lab1 = temp_lab1[temp_lab1['subject_id'].isin(unique_ids)]
    temp_itemid = temp_lab1['itemid']
    
    temp_lab1 = temp_lab1.drop(['itemid','charttime', 'flag','file'], axis=1)
    temp_stats1 = temp_lab1.groupby(['subject_id']).agg(['min', 'max','mean','std'])
    temp_stats1 = temp_stats1.reset_index(drop = True) 
    
    combine = pd.concat([temp_lab1['subject_id'], temp_itemid1, temp_stats1],axis=1, join='inner')
    
    label = prevalence_frame.loc[prevalence_frame['itemid'] == j, 'label'].iloc[0]
    fluid = prevalence_frame.loc[prevalence_frame['itemid'] == j, 'fluid'].iloc[0]
    category = prevalence_frame.loc[prevalence_frame['itemid'] == j, 'category'].iloc[0]
    
    path2 = r'/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/DATA/lab_events_files/{}-{}-{}-{}.csv'.format(label, fluid, category, j)
    combine.to_csv(path2, index=False)
    print(path2)
    

In [ ]:
# Read in all of the lab ids and concat into 1 pandas dataframe
path3 = "/Users/MacUser/Desktop/Fall_2020_Semester/Classes/Precision_Care_Medicine/DATA/lab_events_files/"

cohort_files2 = glob.glob(os.path.join(path3, "*.csv"))
#print(cohort_files2)

# initializing data frame
cohort_df2 = []

# loading all files into data frame
for f2 in cohort_files2:
    df2 = pd.read_csv(f2)
    df2['file'] = f2.split('/')[-1]
    cohort_df2.append(df2)

# concatenating files into single data frame    
cohort2 = pd.concat(cohort_df2, ignore_index=True)
lab_stats = cohort2;
print(lab_stats.head(5))

In [ ]:
# Read in all of the lab ids , normalize + impute + concat into 1 pandas dataframe

path3 = "C:/Users/Safiya/Downloads/lab_events_files"

def normalize_impute(dataset): # normalise and impute nan to mean  function
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))
    dataNorm['subject_id']=dataset['subject_id']
    dataNorm['itemid'] = dataset['itemid']
    dataset = dataNorm
    dataset.apply(pd.to_numeric, errors='ignore')
    dataset = dataset.fillna(dataset.mean())
#same result as sklearn simpleimpute mean strategy 
    dataset['file'] = fname.split('/')[-1]
    print(dataset.head())
    cohort2.append(dataset)
    return dataset

cohort2 =[]
path4 = "C:/Users/Safiya/Downloads/lab_events_files/*.csv"
for fname in glob.glob(path4):
    dataset = pd.read_csv(fname)
    normalize_impute(dataset)
    
cohort2_df = pd.concat(cohort2, ignore_index=True)
lab_stats = cohort2_df;
print(lab_stats)
path5 = "C:/Users/Safiya/Downloads/lab_events_files_normalized/normalized_imputed_lab_stats.csv"
lab_stats.to_csv(path5)